Automatically delint notebooks on save?



In [2]:
// TODO: display a tree of require dependencies and make sure it all still compiles properlyvar importer = require('../Core');var fs = require('fs');var path = require('path');var execSync = require('child_process').execSync;var ncp = require('ncp').ncp;
ncp.limit = 16;
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;var project = path.resolve('../Utilities/.modules');var webstormWin = 'C:\\Program Files\\JetBrains\\WebStorm 2017.1.1\\bin\\format.bat';var webstormMac = '/Applications/WebStorm.app/Contents/bin/format.sh';
var webstorm = '';
if (process.platform === 'win32') {
    webstorm = webstormWin;
} else if (process.platform === 'darwin') {
    webstorm = webstormMac;
}
var idea = path.resolve('../.idea');
$$.async();var diffTwoTexts, cwd = process.cwd();importer.interpret([    'diff'])    .then(r => {
        process.chdir(project);        if (!fs.existsSync(project)) {            fs.mkdirSync(project);        }
        return r;
    })
    .then(r => new Promise((resolve, reject) => {
        // output .idea to set the framework preferences?
        return ncp(idea, project + '/.idea', function (err) {
            if (err) {
                return reject(err);
            }
            resolve(r);
        });
    }))
    .then(r => {        var filename = project + path.basename(r[0].filename) + '-cell-' + r[0].to + '.js';
        fs.writeFileSync(filename, r[0].code);        diffTwoTexts = r[0].runInNewContext();
        // I think I overwrote the default, some way to env variables for this?
        const cmd = '"' + webstorm + '" "' + project
                 + '/" -R -settings "' + project + '/Default.xml"';
        execSync(cmd);
        project.chdir(cwd);        return diffTwoTexts(r[0].code, fs.readFileSync(filename).toString());    })    .then(r => $$.html(r))    .catch(e => $$.sendError(e))

"/Applications/WebStorm.app/Contents/bin/format.sh" "/Users/briancullinan/jupytangular2/Utilities/.modules/" -settings "/Users/briancullinan/jupytangular2/Utilities/.modules/Default.xml"


2017-10-20 21:55:02.948 webstorm[90325:3344226] allVms required 1.8*,1.8+
2017-10-20 21:55:02.952 webstorm[90325:3344238] Value of WEBSTORM_VM_OPTIONS is (null)
2017-10-20 21:55:02.952 webstorm[90325:3344238] Processing VMOptions file at /Users/briancullinan/Library/Preferences/WebStorm2017.2/webstorm.vmoptions
2017-10-20 21:55:02.952 webstorm[90325:3344238] Done
/Applications/WebStorm.app/Contents/bin/idea.properties: 'java.endorsed.dirs = ' already defined in system properties: /Applications/WebStorm.app/Contents/jdk/Contents/Home/jre/lib/endorsed


// npm install diff --savevar JsDiff = require('diff');var Prism = require('prismjs');//require('prismjs/plugins/keep-markup/prism-keep-markup');var htmlEntities = (str) => str.replace(/[\u00A0-\u9999<>\&]/gim, i => '&#' + i.charCodeAt(0) + ';');var diffTwoTexts = (left, right) => { var diff = JsDiff.diffWords(left, right); var code = ''; for (var i = 0; i < diff.length; i++) { if (diff[i].added && diff[i + 1] && diff[i + 1].removed) { var swap = diff[i]; diff[i] = diff[i + 1]; diff[i + 1] = swap; } if (diff[i].removed) { code += '<span class="del">' + htmlEntities(diff[i].value) + '</span>'; } else if (diff[i].added) { code += '<span class="ins">' + htmlEntities(diff[i].value) + '</span>'; } else { code += '<span>' + htmlEntities(diff[i].value) + '</span>'; } } //code = Prism.highlight(code, Prism.languages.javascript) return `<style>.token.comment,.token.prolog,.token.doctype,.token.cdata { color: slategray;}.token.punctuation { color: #999;}.namespace { opacity: .7;}.token.property,.token.tag,.token.boolean,.token.number,.token.constant,.token.symbol,.token.deleted { color: #905;}.token.selector,.token.attr-name,.token.string,.token.char,.token.builtin,.token.inserted { color: #690;}.token.operator,.token.entity,.token.url,.language-css .token.string,.style .token.string { color: #a67f59; background: hsla(0, 0%, 100%, .5);}.token.atrule,.token.attr-value,.token.keyword { color: #07a;}.token.function { color: #DD4A68;}.token.regex,.token.important,.token.variable { color: #e90;}.token.important,.token.bold { font-weight: bold;}.token.italic { font-style: italic;}.token.entity { cursor: help;}.del { text-decoration: none; background: #fadad7;}.ins { background: #eaf2c2; text-decoration: none;}</style><pre>` + code + `</pre>`;}module.exports = diffTwoTexts;diffTwoTexts;

Autofix some linting errors using bslint?

